In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# for repreoducibility
torch.manual_seed(1)

In [4]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [5]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [6]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [7]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
hypothesis = 1 / (1 + torch.exp(-x_train.matmul(W) + b))
# hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [9]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [16]:
losses = -(y_train*torch.log(hypothesis) + (1-y_train)*torch.log(1-hypothesis))
cost = losses.mean()
# F.binary_cross_entropy(hypothesis, y_train)

In [17]:
print(losses)
print(cost)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)
tensor(0.6931, grad_fn=<MeanBackward0>)


In [24]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1)

n_epochs = 1000
for epoch in range(n_epochs + 1):
    
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad() # 이거 안하면, 계속 누적(더하기)함
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, cost.item()))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


### evaluation

In [25]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis)

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward>)


In [28]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:6])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


In [31]:
correct_prediction = prediction.float() == y_train
print(correct_prediction)

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True]])


## 실전

In [35]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
model = BinaryClassifier()
optimizer = optim.SGD(model.parameters(), lr=1)

n_epochs = 100
for epoch in range(n_epochs + 1):
    
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad() # 이거 안하면, 계속 누적(더하기)함
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.float().mean()
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}'.format(
            epoch, n_epochs, cost.item(), accuracy.item() * 100))

Epoch    0/100 Cost: 0.778947 Accuracy 33.33
Epoch   10/100 Cost: 0.606802 Accuracy 66.67
Epoch   20/100 Cost: 0.446548 Accuracy 66.67
Epoch   30/100 Cost: 0.376169 Accuracy 83.33
Epoch   40/100 Cost: 0.318945 Accuracy 83.33
Epoch   50/100 Cost: 0.268428 Accuracy 83.33
Epoch   60/100 Cost: 0.222594 Accuracy 100.00
Epoch   70/100 Cost: 0.183695 Accuracy 100.00
Epoch   80/100 Cost: 0.158160 Accuracy 100.00
Epoch   90/100 Cost: 0.144616 Accuracy 100.00
Epoch  100/100 Cost: 0.134716 Accuracy 100.00
